<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)
!unzip /content/IEMOCAP_splits.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 84.3MB/s]
Archive:  /content/IEMOCAP_splits.zip
   creating: iemocap/
   creating: iemocap/session5/
  inflating: iemocap/session5/test.json  
  inflating: iemocap/session5/train.json  
   creating: iemocap/session2/
  inflating: iemocap/session2/test.json  
  inflating: iemocap/session2/train.json  
   creating: iemocap/session3/
  inflating: iemocap/session3/test.json  
  inflating: iemocap/session3/train.json  
   creating: iemocap/session4/
  inflating: iemocap/session4/test.json  
  inflating: iemocap/session4/train.json  
   creating: iemocap/session1/
  inflating: iemocap/session1/test.json  
  inflating: iemocap/sess

In [7]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/iemocap/session1/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/iemocap/session1/test.json",'r') as f:
    test_data = json.load(f)

In [8]:
train_data

{'Ses02F_impro03_F000': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': '/m/teamwork/t40511_asr/c/

In [9]:
 test_data

{'Ses01F_script02_2_F000': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': '/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_script02_2/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_scrip

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
})

In [42]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap/
    example['wav'] = os.path.join('/content/iemocap/', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

In [43]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [46]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/iemocap/session1/m/teamwork/t40511_asr/c/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'

In [20]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo'])

In [24]:
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')


In [25]:
def update_iemocap_label(label, column="emo"):
    if label == "ang":
        return "anger"
    elif label == "hap":
        return "happiness"
    elif label == "neu":
        return "neutral"
    elif label == "sad":
        return "sadness"

In [26]:
def is_common(emotion):
    if emotion == "neutral" or emotion == "sadness" or emotion == "happiness" or emotion == "anger":
        return 1
    else:
        return 0

In [27]:
def update_iemocap_path(path):
    return "/m/triton/scratch/elec/t405-puhe/p/bijoym1/TempDatasets" + path.split("/m/teamwork/t40511_asr/c")[-1]


In [28]:

def to_label(emo, label2id):
    return label2id[emo]


In [40]:
def iemocap(session):
    train_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/train.json"
    test_json_file = f"/m/triton/scratch/elec/t405-puhe/p/bijoym1/iemocap/session{session}/test.json"

    with open(train_json_file, "r") as f:
        train_data = json.load(f)
    with open( test_json_file, "r") as f:
        test_data = json.load(f)

    train_df = pd.DataFrame.from_dict(train_data, orient="index").reset_index()
    train_df = train_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    test_df = pd.DataFrame.from_dict(test_data, orient="index").reset_index()
    test_df = test_df.rename(columns={'index': 'file_id', 'wav': 'audio'})

    train_df["audio"] = train_df["audio"].apply(update_iemocap_path).values
    test_df["audio"] = test_df["audio"].apply(update_iemocap_path).values

    train_df["emo"] = train_df["emo"].apply(update_iemocap_label)
    test_df["emo"] = test_df["emo"].apply(update_iemocap_label)

    labels = ['anger', 'happiness', 'neutral', 'sadness'] #["ang", "hap", "neu", "sad"]
    label2id, id2label = dict(), dict()

    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label

    # convert categorical label to numerical labels
    train_df["label"] = train_df["emo"].apply(to_label, args=(label2id,))
    test_df["label"] = test_df["emo"].apply(to_label, args=(label2id,))

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']])
    train_audio_data = train_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']])
    test_audio_data = test_audio_data.cast_column("audio", Audio(sampling_rate=16_000))

    ds = DatasetDict({
        'train' : train_audio_data,
        'test' : test_audio_data,
        'dev' : test_audio_data,
    })

    return label2id, id2label, ds

In [35]:
label2id, id2label, ds = iemocap(session=1)

In [36]:

# Now you can access label2id
print(label2id)

{'anger': '0', 'happiness': '1', 'neutral': '2', 'sadness': '3'}


In [37]:
train_audio_data = ds['train']

In [38]:
print(train_audio_data)

Dataset({
    features: ['audio', 'label'],
    num_rows: 4446
})


In [41]:
# Get the audio array and sampling rate for the first example
audio_array = train_audio_data[0]["audio"]["array"]
sampling_rate = train_audio_data[0]["audio"]["sampling_rate"]

print("Audio Array:", audio_array)
print("Sampling Rate:", sampling_rate)

FileNotFoundError: [Errno 2] No such file or directory: '/m/triton/scratch/elec/t405-puhe/p/bijoym1/TempDatasets/IEMOCAP_full_release/Session2/sentences/wav/Ses02F_impro03/Ses02F_impro03_F000.wav'

In [47]:
import torch
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, TrainingArguments, Trainer
!pip install datasets --upgrade

In [49]:
#Define labels and mappings for our dataset
labels = ['anger', 'happiness', 'neutral', 'sadness']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [50]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
# Filter out examples with labels that are not in label2id
dataset = dataset.filter(lambda x: x['emo'] in label2id.keys())
# Preprocess function to convert audio to model-compatible inputs
def preprocess_function(batch):
    # Process audio array into input values for the model
    batch["input_values"] = processor(batch["audio_array"], sampling_rate=16000).input_values[0]
    # Convert label to integer id
    batch["labels"] = label2id[batch["emo"]]
    return batch

# Map preprocess function to the dataset
dataset = dataset.map(preprocess_function, remove_columns=["audio_array", "sample_rate", "wav"])

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Filter:   0%|          | 0/4446 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1085 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1085 [00:00<?, ? examples/s]

ValueError: Column name ['sample_rate', 'audio_array'] not in the dataset. Current columns in the dataset: ['id', 'wav', 'length', 'emo']

In [51]:
import evaluate
accuracy = evaluate.load("accuracy")

In [52]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [53]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer, AutoFeatureExtractor

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
training_args = TrainingArguments(
    output_dir="Imocap_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [length, id, wav, emo]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
#trainer.push_to_hub()

In [59]:

def collate_fn(batch):
batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
inputs = [torch.tensor(example["input_values"]) for example in batch]
labels = [int(example["label"]) for example in batch]
inputs = pad_sequence(inputs, batch_first=True)
return {"input_values": inputs, "label": torch.tensor(labels)}

IndentationError: expected an indented block after function definition on line 1 (<ipython-input-59-d92483c882dd>, line 2)

In [57]:
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()

    total_predictions = 0
    correct_predictions = 0
    total_train_loss = 0

    # train_progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_OF_EPOCHS}, Training", leave=False)

    all_actual_labels = []
    all_predictions = []
    train_confidence_scores = []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        outputs = outputs.logits

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        train_loader.set_postfix(loss=loss.item())

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        total_train_loss += loss.item()

        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

        max_values, max_indices = torch.softmax(outputs, dim=1).max(dim=1)
        probability_label_tuples = [(max_value.item(), max_index.item()) for max_value, max_index in zip(max_values, max_indices)]
        train_confidence_scores.extend(probability_label_tuples)

    total_train_loss /= len(train_loader)
    train_accuracy = correct_predictions / total_predictions

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_actual_labels, all_predictions, average='weighted')

    return (unweighted_recall, weighted_recall, train_accuracy, total_train_loss, train_confidence_scores)
